**Using this template**

1. Change notebook title to assignment title you working on, eg. `Graded Challenge 1 or Milestone 1`
2. Put your identity such as name and batch number below notebook title.
3. Write your description before do any work. State your purpose of this assignment and what you are trying to achieve. 
4. Look at the rubric of this assignment and make sure you understand the criteria.
5. Code your solution in the cell provided below the working area section.
6. State your conlusions, findings, and any other relevant information in the cell provided below the conclusions sections.
7. Save this notebook and rename it to assignment title you working on  eg. `Graded Challenge 1 or Milestone 1`
8. Push your assignment before deadline.
9. Good Luck!

# Graded Challenge 2

1. Name: Jason Rich Darmawan Onggo Putra
2. Batch: 016 RMT

## Description

### Objective

Analysis of the waste management and disposal system in the city of Austin. The analysis result will be the condition of the waste management and disposal system. In addition, the prediction of the weight of waste in the city of Austin on July 10, 2021.

### Problem Statement

#### Problems 1

- [x] What is the average **time between delivery** of every dropoff site? [drop off site AUSTIN IRON AND METAL average load time between delivery is 3 days and 22 hours](#new-drop-off-site).
- [x] What is the **average** waste weight collected of every dropoff site **per delivery**? [answer](#central-tendency)
- [x] What is the **median** waste weight collected of every dropoff site **per delivery**? [answer](#central-tendency)
- [x] What is the **mode** waste weight of every dropoff site **per delivery**? [answer](#central-tendency)

### Problems 2

- [ ] What is the **maximum** waste weight collected of every dropoff site in July 10, 2021?

## Working Area

## Data Retrieve

1. project: `bigquery-public-data`
2. dataset: [austin_waste](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m4!1m3!3m2!1sbigquery-public-data!2saustin_waste)
3. table: [waste_and_diversion](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m5!1m4!4m3!1sbigquery-public-data!2saustin_waste!3swaste_and_diversion)

**The query**:
```
SELECT report_date, load_time, load_type, load_weight, dropoff_site 
FROM `bigquery-public-data.austin_waste.waste_and_diversion`
WHERE report_date BETWEEN '2021-01-01' AND '2021-07-09';
```

## Import Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from enum import Enum
from scipy import stats
import sympy as sy

## Data Preprocessing

Issues:
- [x] column `report_date` dtype is `object`. [Fix](#change-column-report_date-and-load_time-dtype)
- [x] column `load_time` dtype is `object`. [Fix](#change-column-report_date-and-load_time-dtype)
- [x] column `load_weight` have `NaN` rows. [Fix](#data-preprocessing)

In [2]:
df = pd.read_csv("./h8dsft_P0GC2_jason_rich_darmawan_onggo_putra.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26157 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   26157 non-null  object 
 1   load_time     26157 non-null  object 
 2   load_type     26157 non-null  object 
 3   load_weight   25109 non-null  float64
 4   dropoff_site  26157 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1021.9+ KB


In [3]:
df.head()

,report_date,load_time,load_type,load_weight,dropoff_site
0,2021-07-08,2021-07-08 12:00:00 UTC,TIRES,2260.0,MRF
1,2021-07-08,2021-07-08 11:00:00 UTC,TIRES,1260.0,MRF
2,2021-07-08,2021-07-08 03:00:00 UTC,TIRES,800.0,MRF
3,2021-05-01,2021-05-01 12:29:00 UTC,TIRES,760.0,MRF
4,2021-07-03,2021-07-03 12:09:00 UTC,TIRES,1400.0,MRF


In [4]:
df.tail()

,report_date,load_time,load_type,load_weight,dropoff_site
26152,2021-03-01,2021-03-01 11:26:00 UTC,ORGANICS,12100.0,ORGANICS BY GOSH
26153,2021-03-01,2021-03-01 04:27:00 UTC,ORGANICS,10900.0,ORGANICS BY GOSH
26154,2021-03-01,2021-03-01 05:55:00 UTC,ORGANICS,21800.0,ORGANICS BY GOSH
26155,2021-03-01,2021-03-02 01:10:00 UTC,ORGANICS,5640.0,ORGANICS BY GOSH
26156,2021-03-01,2021-03-01 10:22:00 UTC,RECYCLED METAL,360.0,AUSTIN IRON AND METAL


### find dropoff_site that have never filled the `load_weight` column this year

dropoff_site that have never filled the `load_weight` column this year:
- [x] `ONION CREEK`
- [x] `PARK CENTER`
- [x] `ST. EDWARDS`
- [x] `WILCAB`

The decision is to drop these rows because the analyst does not have previous data (of each dropoff_site) to fill the missing value.

In [5]:
df.loc[:,['dropoff_site','load_weight']].groupby(['dropoff_site']).count()

,load_weight
dropoff_site,
AUSTIN IRON AND METAL,50
AUSTIN WOOD RECYCLING,2
BALCONES RECYCLING,3837
BALCONES WOODS,1
GREAT NORTHERN,6
HORNSBY BEND,1652
MRF,168
ON ROUTE HIGH DUMP,7
ONION CREEK,0


In [6]:
for dropoff_site in ['ONION CREEK', 'PARK CENTER', 'ST. EDWARDS', 'WILCAB']:
    print("{0} have at least once filled the `load_weight` column: {1}"
            .format(dropoff_site, df.loc[df['dropoff_site'] == dropoff_site, 'load_weight'].notna().any()))

ONION CREEK have at least once filled the `load_weight` column: False
PARK CENTER have at least once filled the `load_weight` column: False
ST. EDWARDS have at least once filled the `load_weight` column: False
WILCAB have at least once filled the `load_weight` column: False


In [7]:
df2 = df.loc[-df['dropoff_site'].str.contains('ONION CREEK|PARK CENTER|ST. EDWARDS|WILCAB')].copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25267 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   25267 non-null  object 
 1   load_time     25267 non-null  object 
 2   load_type     25267 non-null  object 
 3   load_weight   25109 non-null  float64
 4   dropoff_site  25267 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.2+ MB


### find the dropoff_site that have NaN rows.

- [ ] GREAT NORTHERN have 64 NaN rows
- [ ] HORNSBY BEND have 3 NaN rows
- [ ] ON ROUTE HIGH DUMP have 74 NaN rows
- [ ] ORGANICS BY GOSH have 14 NaN rows
- [ ] TDS LANDFILL have 3 NaN rows

In [8]:
df2.loc[:,'load_weight'].isnull().groupby(df2['dropoff_site']).sum()

dropoff_site
AUSTIN IRON AND METAL     0
AUSTIN WOOD RECYCLING     0
BALCONES RECYCLING        0
BALCONES WOODS            0
GREAT NORTHERN           64
HORNSBY BEND              3
MRF                       0
ON ROUTE HIGH DUMP       74
ORGANICS BY GOSH         14
PARK BEND                 0
STEINER LANDFILL          0
TDS - MRF                 0
TDS LANDFILL              3
WESTFIELD                 0
Name: load_weight, dtype: int64

In [9]:
df3 = df2.copy()

#### fix: dropoff_site TDS LANDFILL

In [10]:
df3_DROPOFF_SITE_TDS_LANDFILL = df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL')].copy()
df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_weight'].isna())]

,report_date,load_time,load_type,load_weight,dropoff_site
15818,2021-06-11,2021-06-11 08:45:00 UTC,SWEEPING,NaN,TDS LANDFILL
22528,2021-01-27,2021-01-27 01:45:00 UTC,SWEEPING,NaN,TDS LANDFILL
24540,2021-02-12,2021-02-12 01:07:00 UTC,GARBAGE COLLECTIONS,NaN,TDS LANDFILL


##### fix: dropoff_site TDS LANDFILL; load_type SWEEPING

The skew value for `load_type` == `SWEEPING` is 0.18. The conclusion is the data distribution is normal. Therefore, the decision is:
1. Find the outliers with `mean +- 3 * std`. Then, take a decision to remove or keep the outliers.
2. Then, fill the NaN rows with the mean value.

In [11]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING = df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'SWEEPING')]
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].skew()

0.18290906329280904

In [12]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].std()
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].std()

df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN + 3 * df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN - 3 * df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD

df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT, df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT

(-9097.925130423664, 18195.850260847328)

There are some outliers found. We will decide whether those outliers are natural variations or a mistake.

In [13]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT) 
                                           | (df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT)]

,report_date,load_time,load_type,load_weight,dropoff_site
1225,2021-03-02,2021-03-02 05:18:00 UTC,SWEEPING,18820.0,TDS LANDFILL
11020,2021-05-06,2021-05-06 03:14:00 UTC,SWEEPING,21300.0,TDS LANDFILL
11021,2021-05-06,2021-05-06 08:52:00 UTC,SWEEPING,20620.0,TDS LANDFILL
11419,2021-05-10,2021-05-10 11:56:00 UTC,SWEEPING,19180.0,TDS LANDFILL
12600,2021-05-18,2021-05-18 01:49:00 UTC,SWEEPING,22760.0,TDS LANDFILL
15412,2021-06-09,2021-06-09 12:52:00 UTC,SWEEPING,18260.0,TDS LANDFILL
18897,2021-07-06,2021-07-06 01:43:00 UTC,SWEEPING,19880.0,TDS LANDFILL
19026,2021-07-07,2021-07-07 12:29:00 UTC,SWEEPING,18800.0,TDS LANDFILL
25517,2021-02-25,2021-02-25 01:33:00 UTC,SWEEPING,19760.0,TDS LANDFILL


We conclude that the outliers are natural variations. So, we will not remove the outliers.

In [14]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].value_counts().head()

11260.0    4
8440.0     3
7380.0     3
14900.0    3
8280.0     3
Name: load_weight, dtype: int64

In [15]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'SWEEPING') & (df3['load_weight'].isna()), 'load_weight'] = \
    df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'SWEEPING'), 'load_weight'].mean()

In [16]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_weight'].isna())]

,report_date,load_time,load_type,load_weight,dropoff_site
24540,2021-02-12,2021-02-12 01:07:00 UTC,GARBAGE COLLECTIONS,NaN,TDS LANDFILL


##### fix: dropoff_site TDS LANDFILL; load_type GARBAGE COLLECTIONS

The skew value for `load_type` == `GARBAGE COLLECTIONS` is 5.56. The conclusion is the data distribution is skewed positively. Therefore, the decision are:

1. Handle the outlier first with Inter-Quartile Range. Then, make the decision to remove or keep the outlierrs.
2. If the skew is STILL above 0,5 or below -0,5. We will fill the NaN rows with the median value.

In [17]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS = df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'GARBAGE COLLECTIONS')]
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,'load_weight'].skew()

5.564210731462809

In [18]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3, df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1 = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,'load_weight'].quantile([.75,.25])
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3 - df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3 + df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR * 1.5
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1 - df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR * 1.5

df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT, df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT

(-4495.0, 39625.0)

There are some outliers found. We will decide whether those outliers are natural variations or not.

In [19]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
                                                      | (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)]
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS') & (df3['load_weight'] == 145160)]
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8204 entries, 565 to 26109
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   8204 non-null   object 
 1   load_time     8204 non-null   object 
 2   load_type     8204 non-null   object 
 3   load_weight   8203 non-null   float64
 4   dropoff_site  8204 non-null   object 
dtypes: float64(1), object(4)
memory usage: 384.6+ KB


We conclude that the outliers are NOT natural variations. So, we will remove the outliers.

In [20]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,['load_weight']].value_counts().head()

load_weight
21780.0        18
25900.0        17
19480.0        16
19120.0        16
24600.0        15
dtype: int64

In [21]:
"""
New knowledge: drop with `DataFrame.drop()`, do not try to replace the value, otherwise you will be left with all NaN rows.
"""
# df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')] = \
#     df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
#                                                           & (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)
#                                                           | (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'].isna())]
df3 = df3.drop(df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL')
                        & (df3['load_type'] == 'GARBAGE COLLECTIONS')
                        & ( (df3['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) | (df3['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT) )
                        & (df3['load_weight'].notna())]
                  .index)

df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')] \
   .loc[(df3['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
        | (df3['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)]

,report_date,load_time,load_type,load_weight,dropoff_site


After removing the outliers, the skew changed to **-0.19**. We canconclude the data distribution is NOW normal. Therefore, we will replace the NaN rows with the mean value.

In [22]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS'), 'load_weight'].skew()

-0.19170335304750802

In [23]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS') & (df3['load_weight'].isna()), 'load_weight'] = \
    df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS'), 'load_weight'].mean()

### We will automate the process **Handle Outliers**

It took way too long to preprocess one dropoff_site. Therefore, we will streamline the process.

pseudo code:
```
def getSkew(dropoff_site: str, load_type: str) -> float: ...

def getMedian(dropoff_site: str, load_type: str) -> float: ...

class STRATEGY(Enum):
    STANDARD_DEVIATION = 0
    IQR = 1

def getUpperLowerLimit(dropoff_site: str, load_type: str, strategy: STRATEGY) -> tuple[float, float]: ...

for dropoff_site in dropoff_sites:
    for load_type in load_types[dropoff_site]:
        load_type_skew = getSkew(dropoff_site, load_type)
        if (load_type_skew < -0.5) | (load_type_skew > 0.5):
            lower_limit, upper_limit = getUpperLowerLimit(dropoff_site, load_type, strategy=STRATEGY.IQR)
            df3 = df3.drop(df3[(df3['dropoff_site'] == dropoff_site) 
                               & (df3['load_type'] == load_type) 
                               & ( (df3['load_weight'] < lower_limit ) | ( df3['load_weight'] > upper_limit ) )
                              ].index)
        else:
            print("dropoff_site {0} load_type {1} have normal data distribution".format(dropoff_site, load_type))
            pass
```

There are some dropoff_site and load_type that still have skewed data distribution after the outliers treament:

dropoff_site:
- **MRF**, with load type:
  - [ ] **TIRES**
- **TDS LANDFILL**, with load type:
  - [ ] **DEAD ANIMAL**
  - [ ] **MIXED LITTER**
  - [ ] **TIRES**
  - [ ] **RECYCLING - SINGLE STREAM**
- **ORGANICS BY GOSH**, with load type:
  - [ ] **DEAD ANIMAL**
- **ON ROUTE HIGH DUMP**, with load type:
  - [ ] **ORGANICS**
- **PARK BEND**, with load type:
  - [ ] **ORGANICS**

TODO: ask Instructor on how to handle skewed data distribution after the first outliers treatment failed.

In [24]:
class DEBUG_STRATEGY(Enum):
    INFO = 0
    VERBOSE = 1

DEBUG = DEBUG_STRATEGY.INFO

df4 = df3.copy()

def getSkew(dropoff_site: str, load_type: str) -> np.float64:
    return df4.loc[(df4['dropoff_site'] == dropoff_site)
                   & (df4['load_type'] == load_type), 'load_weight'] \
              .skew()

class OUTLIER_STRATEGY(Enum):
    IQR = 0

def getLowerUpperLimit(dropoff_site: str, load_type: str, strategy: OUTLIER_STRATEGY) -> tuple[float, float]:
    if strategy == OUTLIER_STRATEGY.IQR:
        Q1, Q3 = df4.loc[(df4['dropoff_site'] == dropoff_site)
                         & (df4['load_type'] == load_type), 'load_weight'] \
                    .quantile([.25, .75])
        IQR = Q3 - Q1
        return Q1 - IQR * 1.5, Q3 + IQR * 1.5
    else:
        raise NotImplementedError("currently just handle skewed data distribution")

getSkew('TDS LANDFILL', 'GARBAGE COLLECTIONS')
getLowerUpperLimit('TDS LANDFILL', 'GARBAGE COLLECTIONS', OUTLIER_STRATEGY.IQR)

for dropoff_site in df4.loc[:,'dropoff_site'].unique():
    for load_type in df4.loc[df4['dropoff_site'] == dropoff_site, 'load_type'].unique():
        dropoff_site_load_type_skew = getSkew(dropoff_site=dropoff_site, load_type=load_type)
        if np.isnan(dropoff_site_load_type_skew):
            """
            load_type_skew with nan value can happen because: there is not enough data, like only 2 rows.
            """
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropping dropoff_site {0} load_type {1} because have nan skew value".format(dropoff_site, load_type))
            df4 = df4.drop(df4.loc[(df4['dropoff_site'] == dropoff_site)
                                   & (df4['load_type'] == load_type)]
                              .index)
        elif (dropoff_site_load_type_skew < -0.5) | (dropoff_site_load_type_skew > 0.5):
            lower_limit, upper_limit = getLowerUpperLimit(dropoff_site=dropoff_site, load_type=load_type, strategy=OUTLIER_STRATEGY.IQR)
            dropoff_site_load_type_rows_length = len(df4.loc[(df4['dropoff_site'] == dropoff_site)
                                                             & (df4['load_type'] == load_type)] \
                                                        .index)
            index_to_drop = df4.loc[(df4['dropoff_site'] == dropoff_site)
                                   & (df4['load_type'] == load_type)
                                   & ( (df4['load_weight'] < lower_limit) | (df4['load_weight'] > upper_limit) )] \
                              .index
            df4 = df4.drop(index_to_drop)
            new_dropoff_site_load_type_skew = df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_type'] == load_type), 'load_weight'].skew()
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropping outliers in dropoff_site {0} load_type {1} because have skewed value: {2:.2f}, outliers: {3:.0%}".format(dropoff_site, 
                                                                                                                                         load_type, 
                                                                                                                                            dropoff_site_load_type_skew, 
                                                                                                                                         len(index_to_drop) / dropoff_site_load_type_rows_length))
            if ((DEBUG == DEBUG_STRATEGY.INFO) | (DEBUG_STRATEGY == DEBUG_STRATEGY.VERBOSE)) & ((new_dropoff_site_load_type_skew < -0.5) | (new_dropoff_site_load_type_skew > 0.5)):
                print("after dropping outliers in dropoff_site {0} load_type {1} still have skewed value: {2:.2f} -> {3:.2f}, outliers: {4:.0%}".format(dropoff_site, 
                                                                                                                                                        load_type, 
                                                                                                                                                        dropoff_site_load_type_skew, 
                                                                                                                                                        new_dropoff_site_load_type_skew,
                                                                                                                                                        len(index_to_drop) / dropoff_site_load_type_rows_length))
        # expected: normal data distribution
        else:
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropoff_site {0} load_type {1} have skew value {2:.2f}. The decision is not to drop the outliers".format(dropoff_site, load_type, dropoff_site_load_type_skew))

after dropping outliers in dropoff_site MRF load_type TIRES still have skewed value: 5.68 -> 0.56, outliers: 3%
after dropping outliers in dropoff_site TDS LANDFILL load_type DEAD ANIMAL still have skewed value: 2.25 -> 0.99, outliers: 8%
after dropping outliers in dropoff_site TDS LANDFILL load_type MIXED LITTER still have skewed value: 1.14 -> 0.63, outliers: 2%
after dropping outliers in dropoff_site TDS LANDFILL load_type TIRES still have skewed value: 3.09 -> -0.92, outliers: 20%
after dropping outliers in dropoff_site TDS LANDFILL load_type RECYCLING - SINGLE STREAM still have skewed value: 0.99 -> 0.99, outliers: 0%
after dropping outliers in dropoff_site ORGANICS BY GOSH load_type DEAD ANIMAL still have skewed value: 3.49 -> 1.47, outliers: 5%
after dropping outliers in dropoff_site ON ROUTE HIGH DUMP load_type ORGANICS still have skewed value: -0.88 -> -0.88, outliers: 0%
after dropping outliers in dropoff_site PARK BEND load_type ORGANICS still have skewed value: 0.54 -> 0.54

### Handle NaN rows.

Assuming we have done the outliers treatment.

~~pseudo code:~~ we only identify 2 combinations, not worth to automate ┗(｀O ´)┛
```
for dropoff_site in dropoff_sites:
    for load_type in dropoff_site['load_type']:
        dropoff_site_load_type_skew = getSkew(dropoff_site, load_type)
        if (dropoff_site_load_type_skew < -0.5) | (dropoff_site_load_type_skew > 0.5):
            # replace NaN with median
        elif:
            # repplace NaN with mean
```

We identified 2 combinations that have NaN rows and have normal data distribution:
- [ ] dropoff_site HORNSBY BEND, load type BRUSH
- [ ] dropoff_site ORGANICS BY GOSH, load type ORGANICS

These combinations are not empty NaN rows. So we will replace it with mean.

In [25]:
for dropoff_site in df4.loc[df4['load_weight'].isna(),'dropoff_site'].unique():
    for load_type in df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_weight'].isna()), 'load_type'].unique():
        dropoff_site_load_type_skew = getSkew(dropoff_site=dropoff_site, load_type=load_type)
        print(f"dropoff_site {dropoff_site} load_type {load_type} skew {dropoff_site_load_type_skew}")
        display(df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_type'] == load_type)])

df5 = df4.copy()

df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
        & (df5['load_type'] == 'BRUSH') 
        & (df5['load_weight'].isna()), 'load_weight'] = df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
                                                                & (df5['load_type'] == 'BRUSH'), 'load_weight'] \
                                                            .mean()

df5.loc[(df5['dropoff_site'] == 'ORGANICS BY GOSH') 
        & (df5['load_type'] == 'ORGANICS') 
        & (df5['load_weight'].isna()), 'load_weight'] = df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
                                                                & (df5['load_type'] == 'BRUSH'), 'load_weight'] \
                                                            .mean()

dropoff_site HORNSBY BEND load_type BRUSH skew 0.2636609436413962


,report_date,load_time,load_type,load_weight,dropoff_site
160,2021-03-20,2021-03-20 07:57:00 UTC,BRUSH,3800.0,HORNSBY BEND
161,2021-03-20,2021-03-20 08:23:00 UTC,BRUSH,4180.0,HORNSBY BEND
162,2021-03-20,2021-03-20 07:11:00 UTC,BRUSH,6500.0,HORNSBY BEND
163,2021-03-20,2021-03-20 10:41:00 UTC,BRUSH,5680.0,HORNSBY BEND
164,2021-03-20,2021-03-20 07:21:00 UTC,BRUSH,3180.0,HORNSBY BEND
...,...,...,...,...,...
26005,2021-03-01,2021-03-01 04:25:00 UTC,BRUSH,6020.0,HORNSBY BEND
26006,2021-03-01,2021-03-01 11:32:00 UTC,BRUSH,8180.0,HORNSBY BEND
26007,2021-03-01,2021-03-01 01:31:00 UTC,BRUSH,2940.0,HORNSBY BEND
26008,2021-03-01,2021-03-01 02:40:00 UTC,BRUSH,6680.0,HORNSBY BEND


dropoff_site ORGANICS BY GOSH load_type ORGANICS skew 0.07768898474656169


,report_date,load_time,load_type,load_weight,dropoff_site
775,2021-02-21,2021-02-21 10:41:00 UTC,ORGANICS,2480.0,ORGANICS BY GOSH
776,2021-03-14,2021-03-14 01:37:00 UTC,ORGANICS,21700.0,ORGANICS BY GOSH
777,2021-03-21,2021-03-21 07:56:00 UTC,ORGANICS,7120.0,ORGANICS BY GOSH
778,2021-02-21,2021-02-21 11:07:00 UTC,ORGANICS,5560.0,ORGANICS BY GOSH
779,2021-03-14,2021-03-14 03:04:00 UTC,ORGANICS,20460.0,ORGANICS BY GOSH
...,...,...,...,...,...
26151,2021-03-01,2021-03-01 11:19:00 UTC,ORGANICS,18580.0,ORGANICS BY GOSH
26152,2021-03-01,2021-03-01 11:26:00 UTC,ORGANICS,12100.0,ORGANICS BY GOSH
26153,2021-03-01,2021-03-01 04:27:00 UTC,ORGANICS,10900.0,ORGANICS BY GOSH
26154,2021-03-01,2021-03-01 05:55:00 UTC,ORGANICS,21800.0,ORGANICS BY GOSH


No more NaN rows.

In [26]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25066 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   25066 non-null  object 
 1   load_time     25066 non-null  object 
 2   load_type     25066 non-null  object 
 3   load_weight   25066 non-null  float64
 4   dropoff_site  25066 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.1+ MB


### Change column `report_date` and `load_time` dtype.

In [27]:
df5['report_date'] = pd.to_datetime(df5['report_date'])
df5['load_time'] = pd.to_datetime(df5['load_time'])

In [28]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25066 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   report_date   25066 non-null  datetime64[ns]     
 1   load_time     25066 non-null  datetime64[ns, UTC]
 2   load_type     25066 non-null  object             
 3   load_weight   25066 non-null  float64            
 4   dropoff_site  25066 non-null  object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(1), object(2)
memory usage: 1.1+ MB


## Data Exploration

### Load Type

In [29]:
df5.loc[:, 'load_type'].unique()

array(['TIRES', 'RECYCLING - SINGLE STREAM', 'BRUSH', 'YARD TRIMMING',
       'BULK', 'ORGANICS', 'DEAD ANIMAL', 'GARBAGE COLLECTIONS',
       'MIXED LITTER', 'SWEEPING', 'LITTER', 'RECYCLED METAL'],
      dtype=object)

### Drop Off Site

In [30]:
df5.loc[:,'dropoff_site'].unique()

array(['MRF', 'TDS - MRF', 'HORNSBY BEND', 'TDS LANDFILL',
       'ORGANICS BY GOSH', 'BALCONES RECYCLING', 'ON ROUTE HIGH DUMP',
       'AUSTIN IRON AND METAL', 'PARK BEND', 'GREAT NORTHERN'],
      dtype=object)

### Central Tendency

There is no one `mode` for rows that is grouped by `dropoff_site`. This is because the column `load_weight` values are mostly unique. If we do `agg([..., pd.Series.mode])`, it will throw error. So, we have to be a bit creative.

The insights for data analyst:
1. drop off site **GREAT NORTHERN** and other drop off site that experience increase in average load weight. This happens because we remove some load type due to:
   1. specific load type only have 2 rows or less. Consequently, we can't calculate the skew value. (+ increase mean)
   2. specific load type with more than 2 rows but the entire rows value are NaN. (+ increase mean)
   3. specific load type have skewed data distribution. Consequently, we remove the outliers with IQR strategy. (- decrease mean)

The insights for management:
1. drop off site **GREAT NORTHERN** have the highest average load weight, 19% higher than the next highest average load weight's dropoff site. After we do outlier handling, the difference increase to 40%.
2. drop off site **AUSTIN IRON AND METAL** have the lowest average load weight.

In [31]:
# force pandas to suppress the scientific notation.
pd.options.display.float_format = '{:,.2f}'.format

df2_central_tendency = df2.loc[:,['dropoff_site', 'load_weight']].groupby(['dropoff_site']).agg([("df2 mean", "mean"), 
                                                                                                 ("df2 median", "median"), 
                                                                                                 ("df2 mode", lambda x: "found {0} different modes".format(len(x.mode().tolist())))])

df5_central_tendency = df5.loc[:,['dropoff_site', 'load_weight']].groupby(['dropoff_site']).agg([("df5 mean","mean"), 
                                                                                                 ("df5 median","median"), 
                                                                                                 ("df5 mode", lambda x: "found {0} different modes".format(len(x.mode().tolist())))])

df2_df5_central_tendency = pd.concat([df2_central_tendency, df5_central_tendency], axis='columns')

# sort the columns
df2_df5_central_tendency_columns = df2_df5_central_tendency.columns.tolist()
df2_df5_central_tendency_columns = df2_df5_central_tendency_columns[0:1] \
                                   + df2_df5_central_tendency_columns[3:4] \
                                   + df2_df5_central_tendency_columns[1:2] \
                                   + df2_df5_central_tendency_columns[4:5] \
                                   + df2_df5_central_tendency_columns[2:3] \
                                   + df2_df5_central_tendency_columns[5:6]
df2_df5_central_tendency = df2_df5_central_tendency[df2_df5_central_tendency_columns]

display(df2_df5_central_tendency)

pd.options.display.float_format = None

load_weight                                  \
                         df2 mean  df5 mean df2 median df5 median   
dropoff_site                                                        
AUSTIN IRON AND METAL      451.20    426.25     400.00     395.00   
AUSTIN WOOD RECYCLING    4,050.00       NaN   4,050.00        NaN   
BALCONES RECYCLING       9,464.65  9,464.65   9,840.00   9,840.00   
BALCONES WOODS           8,460.00       NaN   8,460.00        NaN   
GREAT NORTHERN          19,153.33 21,244.00  21,110.00  21,320.00   
HORNSBY BEND             6,628.64  6,624.97   6,390.00   6,380.00   
MRF                      1,799.98  1,426.48   1,360.00   1,330.00   
ON ROUTE HIGH DUMP      12,525.71 12,525.71  14,940.00  14,940.00   
ORGANICS BY GOSH        11,818.48 11,806.21  11,920.00  11,900.00   
PARK BEND               13,385.00 13,385.00  12,330.00  12,330.00   
STEINER LANDFILL           780.00       NaN     780.00        NaN   
TDS - MRF                9,872.55  9,875.78  10,330.00  10,340.00   
TDS LANDFILL            15,169.41 15,118.93  14,920.00  14,940.00   
WESTFIELD               16,020.00       NaN  16,020.00        NaN   

                                                                         
                                      df2 mode                 df5 mode  
dropoff_site                                                             
AUSTIN IRON AND METAL  found 3 different modes  found 3 different modes  
AUSTIN WOOD RECYCLING  found 2 different modes                      NaN  
BALCONES RECYCLING     found 1 different modes  found 1 different modes  
BALCONES WOODS         found 1 different modes                      NaN  
GREAT NORTHERN         found 6 different modes  found 5 different modes  
HORNSBY BEND           found 1 different modes  found 1 different modes  
MRF                    found 1 different modes  found 1 different modes  
ON ROUTE HIGH DUMP     found 7 different modes  found 7 different modes  
ORGANICS BY GOSH       found 1 different modes  found 1 different modes  
PARK BEND              found 4 different modes  found 4 different modes  
STEINER LANDFILL       found 1 different modes                      NaN  
TDS - MRF              found 1 different modes  found 1 different modes  
TDS LANDFILL           found 3 different modes  found 3 different modes  
WESTFIELD              found 1 different modes                      NaN

### Variance

In this part, Jason chose the drop off site **AUSTIN IRON AND METALn** to analyze further. This is because it has the lowest variance (of groupby drop off site). It also have the least amount of load types. Hopefully, we can predict accurately the load weight for load type **RECYCLED METAL** on July 10, 2021.

The insights to management:
1. High variance means for each delivery, the load weight have high difference. For example, yesterday drop off site **AUSTIN IRON AND METAL** collected 1.500 recycled metal. Today, they collected 10.000 recycled metal. This makes predicting the future harder.
2. From standard deviation, we can expect the minimum and maximum load weight per delivery. For example, if the average load weight of recycled metal is 50. The standard deviation is 5. Then we can expect today's load weight to be around `50 +- 5` (or between 45 to 55 recycled metal)
3. The range is just a historical data. It tells us the minimum and maximum load weight for a specific drop off site and load type.

In [32]:
# force pandas to suppress the scientific notation.
pd.options.display.float_format = '{:,.2f}'.format

display(df5.groupby(['dropoff_site'])['load_weight'].agg(['var', 
                                                          'std',
                                                          ('range', lambda x: "{0:,.2f} - {1:,.2f}".format(x.min(), x.max())),
                                                          'count']))

display(df5.groupby(['dropoff_site', 'load_type'])['load_weight'].agg(['var', 
                                                                       'std',
                                                                       ('range', lambda x: "{0:,.2f} - {1:,.2f}".format(x.min(), x.max())),
                                                                       'count']))

pd.options.display.float_format = None

,var,std,range,count
dropoff_site,,,,
AUSTIN IRON AND METAL,"32,585.64",180.51,140.00 - 830.00,48
BALCONES RECYCLING,"14,916,926.03","3,862.24","140.00 - 25,780.00",3837
GREAT NORTHERN,"7,634,080.00","2,762.98","17,160.00 - 24,880.00",5
HORNSBY BEND,"10,474,661.65","3,236.46","60.00 - 20,460.00",1652
MRF,"756,077.76",869.53,"40.00 - 4,120.00",160
ON ROUTE HIGH DUMP,"25,274,361.90","5,027.36","4,060.00 - 18,460.00",7
ORGANICS BY GOSH,"25,718,646.21","5,071.36","20.00 - 29,660.00",5751
PARK BEND,"32,736,366.67","5,721.57","8,480.00 - 20,400.00",4
TDS - MRF,"16,640,346.15","4,079.26","40.00 - 29,940.00",2945


var      std  \
dropoff_site          load_type                                          
AUSTIN IRON AND METAL RECYCLED METAL                32,585.64   180.51   
BALCONES RECYCLING    RECYCLING - SINGLE STREAM 14,916,926.03 3,862.24   
GREAT NORTHERN        GARBAGE COLLECTIONS        7,634,080.00 2,762.98   
HORNSBY BEND          BRUSH                      7,727,648.71 2,779.86   
                      YARD TRIMMING             20,280,793.43 4,503.42   
MRF                   TIRES                        756,077.76   869.53   
ON ROUTE HIGH DUMP    ORGANICS                  25,274,361.90 5,027.36   
ORGANICS BY GOSH      DEAD ANIMAL                   21,371.24   146.19   
                      ORGANICS                  25,363,827.47 5,036.25   
                      RECYCLED METAL            41,652,720.00 6,453.89   
PARK BEND             ORGANICS                  32,736,366.67 5,721.57   
TDS - MRF             GARBAGE COLLECTIONS       53,283,121.16 7,299.53   
                      RECYCLING - SINGLE STREAM 15,844,366.84 3,980.50   
TDS LANDFILL          BULK                      15,220,546.84 3,901.35   
                      DEAD ANIMAL                    7,892.89    88.84   
                      GARBAGE COLLECTIONS       48,916,474.77 6,994.03   
                      LITTER                     1,947,616.73 1,395.57   
                      MIXED LITTER               2,082,024.42 1,442.92   
                      ORGANICS                       5,564.71    74.60   
                      RECYCLING - SINGLE STREAM  6,254,800.00 2,500.96   
                      SWEEPING                  20,577,778.75 4,536.27   
                      TIRES                        160,428.57   400.54   

                                                                 range  count  
dropoff_site          load_type                                                
AUSTIN IRON AND METAL RECYCLED METAL                   140.00 - 830.00     48  
BALCONES RECYCLING    RECYCLING - SINGLE STREAM     140.00 - 25,780.00   3837  
GREAT NORTHERN        GARBAGE COLLECTIONS        17,160.00 - 24,880.00      5  
HORNSBY BEND          BRUSH                          60.00 - 19,660.00   1515  
                      YARD TRIMMING               1,220.00 - 20,460.00    137  
MRF                   TIRES                           40.00 - 4,120.00    160  
ON ROUTE HIGH DUMP    ORGANICS                    4,060.00 - 18,460.00      7  
ORGANICS BY GOSH      DEAD ANIMAL                       20.00 - 540.00     18  
                      ORGANICS                       20.00 - 29,660.00   5728  
                      RECYCLED METAL              5,160.00 - 20,760.00      5  
PARK BEND             ORGANICS                    8,480.00 - 20,400.00      4  
TDS - MRF             GARBAGE COLLECTIONS         3,400.00 - 29,940.00     28  
                      RECYCLING - SINGLE STREAM      40.00 - 27,060.00   2917  
TDS LANDFILL          BULK                           20.00 - 28,000.00   1590  
                      DEAD ANIMAL                       20.00 - 380.00    121  
                      GARBAGE COLLECTIONS             0.12 - 35,900.00   8197  
                      LITTER                         600.00 - 6,340.00     39  
                      MIXED LITTER                     0.00 - 7,000.00    321  
                      ORGANICS                          20.00 - 220.00     18  
                      RECYCLING - SINGLE STREAM      420.00 - 5,320.00      3  
                      SWEEPING                      760.00 - 22,760.00    360  
                      TIRES                        1,300.00 - 2,380.00      8

### New Drop Off Site

The sample data that we are going to use is drop off site AUSTIN IRON AND METAL with load type RECYCLED METAL.

H0: μ <= 356

H1: μ > 356

The usual confidence interval is 95%. So we will use it. Therefore, we will reject the null hypothesis if the p-value is less than 0.05.

In [33]:
df5_AUSTIN_IRON_AND_METAL = df5.loc[df5['dropoff_site'] == 'AUSTIN IRON AND METAL']

print("drop off site AUSTIN IRON AND METAL load type RECYCLED METAL estimated standard deviation population:", df5_AUSTIN_IRON_AND_METAL.loc[:,'load_weight'].std(ddof=0))

drop off site AUSTIN IRON AND METAL load type RECYCLED METAL estimated standard deviation population: 178.62466468361342


Since our p-value is less than `0.05`, so we reject the null hypothesis and we can conclude that, the average load weight of drop off site is more than 356 recycled metal per delivery.

In [34]:
statistic, pvalue = stats.ttest_1samp(a=df5_AUSTIN_IRON_AND_METAL.loc[:,'load_weight'], 
                                      popmean=356,
                                      alternative="greater")
statistic, pvalue

(2.696210719450297, 0.004851352331943044)

The average load time between delivery is 3 days 22 hours. Therefore, if drop off site AUSTIN IRON AND METAL were to be closed off and if the new drop off site can process `356 + 3 * 178.6` recycled materials `(from estimated mean population + 3 * estimated standard deviation population)` recycled metal every 3 days, we can safely assume the new drop off site with capacity of `891.8` reycycled metal every 3 days can operate without issue 99.73% of the time.

In [35]:
df6 = df5.sort_values(['load_time']).copy()
df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL','previous_load_time'] = df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL','load_time'].shift()
df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL','between_load_time'] = df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL','load_time'] - df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL','previous_load_time']
df6.loc[df6['dropoff_site'] == 'AUSTIN IRON AND METAL', 'between_load_time'].mean()

Timedelta('3 days 21:57:05.106382978')

### Forecasting July 10th, 2021

In [46]:
# force pandas to supress the scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# groupby automatically sort the column, so we do not need to call `sort_values('report_date')`
# w = f(t)
# dw
# -- = f'(t)
# dt
y = df6.loc[:,['report_date', 'load_weight']].groupby(['report_date']).sum()

# we calculate the cumulative load weight because
# 1. drop off site is a depot to collect waste, not to send it somewhere else.
# 2. no drop off site have ever write negative load_weight / 
#    indicating they send the waste out from the drop off site.
# 3. if we imagine `w` to have a function e.g `f(t)`, we don't know what's on the right side of the equation,
#    but let's say we do, for example f(2) should be = w on time 1 + w on time 2.
#    therefore, the derivative of f'(t) or v should be cumulative as well.
y['cumulative_load_weight'] = y['load_weight'].cumsum()

dx = [0]

for i in range(1, len(y.reset_index().index)):
    dx.append(dx[i-1] + i)

v = np.diff(y.loc[:,'load_weight']) / np.diff(dx)

w_besok = y.iloc[-1]['load_weight'] + v[-1] * 1
print("whari ini: {0:,.2f}, wbesok: {1:,.2f}".format(y.iloc[-1]['load_weight'], w_besok))

whari ini: 120,340.00, wbesok: 112,429.57


# Problem 1

Answering the questions:
1. We use 2 strategy to handle outliers.
   1. If the data distribution is normal, we can safely use the `sample mean +- 3 * sample standard deviation` as the upper and lower limit. Data outside the upper and lower limit will be considered as outliers. However, because we do automatic strategy and so there is no time to determine whether the outliers are natural variations or not, we decided not to drop the outliers if the data distribution is normal.
   2. If the data distribution is skewed, we will use Interquartile Range (IQR) to detect outliers. In this case, lower limit `Q1 - IQR * 1.5` and upper limit `Q3 + IQR * 1.5`. Any data outside the lower and uppepr limit will be deleted.
   3. The reason we use IQR for data that have skewed distribution is because of the very nature of `mean` formula which is `sum of the data divided by length of the data`. For example, if we have a data of `[1,1,1,100]`, at a glance we can determine that 100 is an outlier. If we use the `sample mean +- 3 * sample standard deviation` strategy, the result is that no data is detected as outliers since the lower limit is `-122.75` and upper limit is `174.5`. This where IQR comes in handy, it is not as affected by outliers as the previous strategy. With IQR, the lower limit is `-36.125` and upper limit is `62.875`. The result is data without outlier `[1,1,1]`

2. I will explain with example, 5% confidence interval, the usual number we use, if we do reject H0, we can confidently says that there is only 5% chance of the alternative hypothesis is failing, while 95% of the time the hypothesis alternative is true.

3. I use hypothesis testing one sample right sided because I want to confidently know the minimum population mean, from there I can estimate the minimum capacity of a drop off site to be able to work without issue 99.73% of the time.

# Problem 2

Answering questions:
1. For this case, we use the numeric method. This is easier because we can derivate from `load_weight` to `load_weight speed` of each day just by calculating the difference between `load_weight` index N+1 substracted by `load_weight` index N.
2. function `numpy.diff()` is actually calculating the difference between value N+1 and value N. This is because we are using the numeric method. Therefore, we need `mathemtical function` to calculate the derivative.
3. The estimated total load weight on July 10th, 2021 is 112,429.57

# Conclusions, Assumptions, Overall Analysis

According to the sample data after we clean up the outliers, these are the insight for the management about drop off site AUSTIN IRON AND METAL:
1. The average load time between delivery is 3 days and 22 hours. This means, for every delivery, the drop off site have exactly 3 days and 22 hours to process the recycled metal before the next delivery coming in.
2. The hypothesis testing one sample one sided concludes that the estimated population mean is larger than 356. Therefore, if we close off AUSTIN IRON AND METAL and make a new drop off site. We can safely design a drop off site with capacity of `891.8` recycled metal every 3 days that will operate without issue 99,73% of the time.

Other insight for the management is:
1. The estimated total load weight on July 10th, 2021 is 112,429.57. This means if we combine load weight collected of every drop off sites in Austin, the estimated total load weight will be 112,429.57 tons (the load weight measurement is taken from [Austin, Texas government website](https://data.austintexas.gov/Utilities-and-City-Services/Waste-Collection-Diversion-Report-daily-/mbnu-4wq9))